# 01 - Training, Validation, Testing

We are at the moment only testing our accuracy on our test data after we submit it to Kaggle.

Submissions to Kaggle are limited, so we should rather try to test on our training data already.

What we want is to split the train data into:

1. a set we want to train on
2. a set we use only for validating our training

Let's start out with a validation set of size  **1/3**

In [59]:
import tools.extract as e
import tools.transform as t

train, test = e.load("data")

DATA_SIZE = len(train)
VALIDATION_SIZE = int(len(train) * 1/8)
TRAIN_SIZE = DATA_SIZE - VALIDATION_SIZE

from sklearn.utils import shuffle

shuffled = shuffle(train)

validation_set = shuffled [ TRAIN_SIZE: ]
training_set = shuffled [ :TRAIN_SIZE ]

In [60]:
import tools.extract as e
import tools.transform as t

# 'PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
# 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'

features = ['Pclass', 'Sex', 'Age']

def transform(data):
    selected = t.select(data, features)
    selected["Sex"], _ = ( t.encode_categories( selected, "Sex" ))
    selected["Age"] = t.set_NaN(selected, "Age", 0)
    return selected

import tools.classifiers.decision_tree as dt
import tools.predict as p

labels = training_set['Survived']

tree = dt.DecisionTree(features)
p.train(tree, transform(training_set), labels)
predictions = p.predict(tree, transform(validation_set))

ID = "PassengerId"
LABEL = "Survived"

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [70]:
def accuracy( predicted, ground_truth ):
    correct = (predicted == ground_truth).sum()
    print( "correct/total: " + str(correct) +"/"+ str(len(predicted)))
    print( "accuracy: " + str(correct/len(predicted)) )

accuracy(predictions, validation_set['Survived'])


correct/total: 86/111
accuracy: 0.774774774775


## Cool! Now we are set to rumble!

Let us try different depth limitations for the tree